In [1]:
import stanfordnlp
stanfordnlp.download('uk')

Using the default treebank "uk_iu" for language "uk".
Would you like to download the models for: uk_iu now? (Y/n)
Y

Default download directory: /Users/oleh.palianytsia/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models.zip


100%|██████████| 234M/234M [01:13<00:00, 3.16MB/s] 



Download complete.  Models saved to: /Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models.zip
Extracting models file for: uk_iu
Cleaning up...Done.


In [2]:
nlp = stanfordnlp.Pipeline(lang='uk')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models/uk_iu_tokenizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models/uk_iu_tagger.pt', 'pretrain_path': '/Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models/uk_iu.pretrain.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models/uk_iu_lemmatizer.pt', 'lang': 'uk', 'shorthand': 'uk_iu', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/oleh.palianytsia/stanfordnlp_resources/uk_iu_models/uk_iu_parser.pt', 'pretrain_path': '/Users/ole

In [3]:
# виклик nlp(<text>) друкує ворнінг, тому вимикаємо ворнінги
import warnings
warnings.filterwarnings('ignore')

In [14]:
cats = nlp("Я люблю товстеньких котів і маленьких мишей")
cats.sentences[0].print_dependencies()
print('')
cats.sentences[0].print_tokens()

('Я', '2', 'nsubj')
('люблю', '0', 'root')
('товстеньких', '4', 'amod')
('котів', '2', 'obj')
('і', '7', 'cc')
('маленьких', '7', 'amod')
('мишей', '4', 'conj')

<Token index=1;words=[<Word index=1;text=Я;lemma=я;upos=PRON;xpos=Pp-1-ysnn;feats=Animacy=Anim|Case=Nom|Number=Sing|Person=1|PronType=Prs;governor=2;dependency_relation=nsubj>]>
<Token index=2;words=[<Word index=2;text=люблю;lemma=любити;upos=VERB;xpos=Vmpip1s;feats=Aspect=Imp|Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin;governor=0;dependency_relation=root>]>
<Token index=3;words=[<Word index=3;text=товстеньких;lemma=товстенький;upos=ADJ;xpos=Ao--pafy;feats=Animacy=Anim|Case=Acc|Number=Plur;governor=4;dependency_relation=amod>]>
<Token index=4;words=[<Word index=4;text=котів;lemma=кіт;upos=NOUN;xpos=Ncmpay;feats=Animacy=Anim|Case=Acc|Gender=Masc|Number=Plur;governor=2;dependency_relation=obj>]>
<Token index=5;words=[<Word index=5;text=і;lemma=і;upos=CCONJ;xpos=Ccs;feats=_;governor=7;dependency_relation=cc>]>
<Token in

In [5]:
with open("../../../tasks/02-structural-linguistics/data/tyhrolovy.txt") as f:
    tyhrolovy = f.read()

In [6]:
tyhrolovy_doc = nlp(tyhrolovy)

In [15]:
def noun_adj_freqs(doc):
    freqs = {}

    for sent in doc.sentences:
        for word in sent.words:
            if word.upos == 'ADJ' and word.dependency_relation == 'amod':
                governor = sent.words[word.governor - 1]
                if governor.upos in ['NOUN', 'PROPN'] and 'Animacy=Anim' in governor.feats:
                    noun_lemma = governor.lemma
                    adj_lemma = word.lemma
                    adj_freqs = freqs.setdefault(noun_lemma, {})
                    adj_freqs[adj_lemma] = adj_freqs.get(adj_lemma, 0) + 1
                
    return freqs

In [16]:
noun_adj_freqs(cats)

{'кіт': {'товстенький': 1}, 'миша': {'маленький': 1}}

In [17]:
def display_freqs(freqs):
    tuples = [(noun, adj, freq) for noun, d in freqs.items() for adj, freq in d.items()]
    for noun, adj, freq in sorted(tuples, key=lambda item: item[2], reverse=True):
        print("{} - {}: {}".format(noun, adj, freq))

In [18]:
display_freqs(noun_adj_freqs(cats))

кіт - товстенький: 1
миша - маленький: 1


In [19]:
tyhrolovy_freqs = noun_adj_freqs(tyhrolovy_doc)

In [20]:
display_freqs(tyhrolovy_freqs)

Сірко - старий: 81
мороз - старий: 7
сірчихий - старий: 6
начальник - великий: 5
зять - цибатий: 5
зайчик - сонячний: 4
коза - дикий: 4
пес - якутський: 3
пес - великий: 3
звірок - смугастий: 3
дівчина - химерний: 3
Сірка - старий: 3
тварина - гордий: 3
сірко - старий: 3
пан - великий: 3
товариш - рідний: 2
людина - різний: 2
людина - добрий: 2
людина - вимучений: 2
злочинець - державний: 2
кішка - зв’язаний: 2
олень - молодий: 2
звір - хутряний: 2
якут - бронзовий: 2
пес - гостровухий: 2
пес - вірний: 2
мама - вража: 2
турок - владивостоцький: 2
бог - дивний: 2
бог - волохатий: 2
бог - Могутній: 2
рись - туя: 2
мандрівник - змучений: 2
кінь - сідланий: 2
дівчина - вража: 2
дівчина - розмальований: 2
собака - звіровий: 2
козак - уссурійський: 2
мисливаць - справжній: 2
коній - нав’ючений: 2
оленя - плямистий: 2
друг - старий: 2
друг - вірний: 2
Мороза - старий: 2
Григорій - приголомшений: 2
тигра - житий: 2
дракон - легендарний: 1
дракон - реальний: 1
дракон - справжній: 1
дракон - най